# Project 1: An Analysis of Maternal Health Outcomes in America
## by Group 7: Mia Metni, Khadija Collymore and Christopher Wiggs
## 1. Research Question
### Are maternal mortality rates of women in America highest among black women (as opposed to women with other racial backgrounds)?

## 2. Hypothesis
### The elevated maternal mortality rates among black women in the United States compared to other racial groups stem from a complex interplay of factors:
- healthcare disparities, 
- socio-economic factors, 
- systemic biases, 
- limited access to quality healthcare, 
- economic inequality, 
- racial biases within healthcare systems 



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from scipy.stats import linregress